In [1]:
import sys
sys.path.insert(0, '../') 

In [2]:
import rdflib
from wlkernel import *
import random
from itertools import chain
from pathlib import Path
from pprint import pprint
from collections import Counter

In [3]:
rdf_graph = rdflib.Graph().parse('../tests/resources/aifbfixed_complete.n3', format='n3')

In [4]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
all_instances = [(t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) )]
assert len(all_instances) == len(set(all_instances))

In [5]:
n = 20
random.seed(42)
instances = random.sample(all_instances, n)
instances_id = [x[0] for x in instances]
instances_class = [x[1] for x in instances]
# print('Amount of the values of the classes:\n\t')
# pprint(Counter(instances_class).most_common())

In [6]:
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')
rdf_graph.remove( (None, member_uri, None) )
rdf_graph.remove( (None, affiliation_uri, None) )

In [7]:
instances_id = [str(x) for x in instances_id]
instances_class = [str(x) for x in instances_class]

In [10]:
kernel_matrix = compute_kernel_matrix(rdf_graph, instances_id, 2, 1)

KeyboardInterrupt: 

In [20]:
kernel_matrix_par = compute_kernel_matrix_par(rdf_graph, instances_id, 2, 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 187 out of 190 | elapsed:  2.8min remaining:    2.7s
[Parallel(n_jobs=-1)]: Done 190 out 

In [23]:
kernel_matrix == kernel_matrix_par

True

In [23]:
X = kernel_matrix_par
y = instances_class

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
tree_clf = RandomForestClassifier(n_estimators=75)
tree_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [2]:
y_pred_train = tree_clf.predict(X_train)
y_pred = tree_clf.predict(X_test)

NameError: name 'tree_clf' is not defined

In [28]:
def accuracy(y_true, y_pred): return sum([x == y for x, y in zip(y_true, y_pred)]) / len(y_true)

In [1]:
print('Train accuracy: ', accuracy(y_train, y_pred_train))
print('Test accuracy: ', accuracy(y_test, y_pred))

NameError: name 'accuracy' is not defined

In [30]:
y_pred

array(['http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance'],
      dtype='<U86')